In [1]:
import kuzu

In [2]:
DB_NAME = "./db"
db = kuzu.Database(DB_NAME)
conn = kuzu.Connection(db)

In [3]:
conn.execute(
    """
    CREATE NODE TABLE Person (
        id STRING,
        name STRING,
        age INT64,
        PRIMARY KEY (id)
    )
    """
)

conn.execute(
    """
    CREATE NODE TABLE City (
        id STRING,
        city STRING,
        PRIMARY KEY (id)
    )
    """
)

conn.execute(
    """
    CREATE NODE TABLE Company (
        id STRING,
        company STRING,
        PRIMARY KEY (id)
    )
    """
)

## Create relationship tables
The following cells create the required relationship tables.

In [4]:
conn.execute(
    """
    CREATE REL TABLE LivesIn (
        FROM Person
        TO City
    )
    """
)

conn.execute(
    """
    CREATE REL TABLE HeadquarteredIn (
        FROM Company
        TO City
    )
    """
)

conn.execute(
    """
    CREATE REL TABLE WorksAt (
        FROM Person
        TO Company
    )
    """
)

In [5]:
# Ingest node data
conn.execute("COPY Person FROM 'data/person.csv'")
conn.execute("COPY City FROM 'data/city.csv'")
conn.execute("COPY Company FROM 'data/company.csv'")

In [6]:
# Ingest relationship data
conn.execute("COPY LivesIn FROM 'data/lives_in.csv'")
conn.execute("COPY HeadquarteredIn FROM 'data/headquartered_in.csv'")
conn.execute("COPY WorksAt FROM 'data/works_at.csv'")

In [7]:
# In which cities do employees most commonly live?
result = conn.execute(
    """
    MATCH (p:Person)-[:LivesIn]->(ci:City)
    RETURN ci.city AS city_name, count(p) AS num_persons
    """
)
while result.has_next():
    print(result.get_next())

['San Francisco', 2]
['Charlotte', 2]


In [8]:
# How many people live and work in the city of their company's headquarters?
result = conn.execute(
    """
    MATCH (c1:City)<-[:LivesIn]-(p:Person)-[:WorksAt]->(:Company)-[:HeadquarteredIn]->(c2:City)
    WHERE c1.city = c2.city
    RETURN COUNT(p) AS num_persons
    """
)
while result.has_next():
    print(result.get_next()[0])

1


In [9]:
# Does a given person have a path to a given company?
query = """
    MATCH (p:Person)-[*1..2]-(q:Person)-[:WorksAt]->(co:Company)
    WHERE p.name = "James" AND co.company = "Apple"
    RETURN DISTINCT q.name AS person
"""
result = conn.execute(query)
while result.has_next():
    print(result.get_next()[0])

Lisa
